In [1]:
# Suppress distracting outputs in this notebook
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger('esmvalcore')
logger.setLevel(logging.WARNING)

![image](https://github.com/eWaterCycle/ewatercycle/raw/main/docs/examples/logo.png)

# User guide

This user manual will explain how the eWaterCycle Python package can be used to perform hydrological experiments. We will walk through the following chapters:

- parameter sets
- forcing data
- model instances
- using observations
- analysis

Each of these chapters correspond to a so-called "subpackage" of eWaterCycle Python package. Before we continue, however, we will briefly explain the configuration file.

**Configuration**

To be able to find all needed data and models eWaterCycle comes with a configuration object. This configuration contains system settings for eWaterCycle (which container technology to use, where is the data located, etc). In general these should not need to be changed by the user for a specific experiment, and ideally a user would never need to touch this configuration on a properly managed system. However, it is good to know that it is there. 

You can see the default configuration on your system like so:

In [2]:
from ewatercycle import CFG
CFG

Config({'container_engine': 'singularity',
        'esmvaltool_config': None,
        'ewatercycle_config': PosixPath('/home/peter/.config/ewatercycle/ewatercycle.yaml'),
        'grdc_location': None,
        'output_dir': PosixPath('/home/peter/ewatercycle/ewatercycle/docs/examples'),
        'parameter_sets': {'lisflood_fraser': {'config': 'lisflood_fraser/settings_lat_lon-Run.xml',
                                               'directory': 'lisflood_fraser',
                                               'doi': 'N/A',
                                               'supported_model_versions': {'20.10'},
                                               'target_model': 'lisflood'},
                           'pcrglobwb_rhinemeuse_30min': {'config': 'pcrglobwb_rhinemeuse_30min/setup_natural_test.ini',
                                                          'directory': 'pcrglobwb_rhinemeuse_30min',
                                                          'doi': 'https://doi.org/10.52

Note: a path on the local filesystem is always denoted as "dir" (short for directory), instead of folder, path, or location. Especially location can be confusing in the context of geospatial modeling.

It is also possible to store and load custom configuration files. For more information, see [system setup](https://ewatercycle.readthedocs.io/en/latest/system_setup.html#configure-ewatercycle)

## Parameter sets

Parameter sets are an essential part of many hydrological models, and for the eWaterCycle package as well.

In [3]:
import ewatercycle.parameter_sets

The default [system setup](https://ewatercycle.readthedocs.io/en/latest/system_setup.html#download-example-parameter-sets) includes a number of example parameter sets that can be used directly. System administrators can also add available parameter sets that are globally availble to all users. In the future, we're hoping to add functionality to fetch new parameter sets using a DOI as well.

To see the available parameter sets:

In [4]:
ewatercycle.parameter_sets.available_parameter_sets()

('lisflood_fraser', 'pcrglobwb_rhinemeuse_30min', 'wflow_rhine_sbm_nc')

Since most parameter sets are model specific, you can filter the results as well:

In [5]:
ewatercycle.parameter_sets.available_parameter_sets(target_model='wflow')

('wflow_rhine_sbm_nc',)

Once you have found a suitable parameter set, you can load it and see some more details:

In [6]:
parameter_set = ewatercycle.parameter_sets.get_parameter_set('wflow_rhine_sbm_nc')
print(parameter_set)

Parameter set
-------------
name=wflow_rhine_sbm_nc
directory=/home/peter/ewatercycle/ewatercycle/docs/examples/parameter-sets/wflow_rhine_sbm_nc
config=/home/peter/ewatercycle/ewatercycle/docs/examples/parameter-sets/wflow_rhine_sbm_nc/wflow_sbm_NC.ini
doi=N/A
target_model=wflow
supported_model_versions={'2020.1.1'}


or you can access individual attributes of the parameter sets

In [7]:
parameter_set.supported_model_versions

{'2020.1.1'}

Should you wish to configure your own parameter set (e.g. for PCRGlobWB in this case), this is also possible:

In [8]:
custom_parameter_set = ewatercycle.parameter_sets.ParameterSet(
    name="custom_parameter_set",
    directory="/home/peter/ewatercycle/ewatercycle/docs/examples/parameter-sets/pcrglobwb_rhinemeuse_30min",
    config="/home/peter/ewatercycle/ewatercycle/docs/examples/parameter-sets/pcrglobwb_rhinemeuse_30min/setup_natural_test.ini",
    target_model="pcrglobwb",
    doi="https://doi.org/10.5281/zenodo.1045339",
    supported_model_versions={'setters'}
)

<div class="alert alert-info">

As you can see, an eWaterCycle parameter set is defined fully by a directory and a configuration file. The configuration file typically informs the model about the structure of the parameter set (e.g. "what is the filename of the land use data"). It is possible to change these settings later, when [setting up the model](#Models).

</div>

## Forcing data

eWaterCycle can load or generate forcing data for a model using the `forcing` module. 

In [2]:
import ewatercycle.forcing

### Existing forcing from external source

We first show how existing forcing data can be loaded with eWaterCycle. The wflow example parameter set already includes forcing data that was generated manually by the scientists at Deltares.

In [10]:
forcing = ewatercycle.forcing.load_foreign(
    directory = parameters.directory,
    target_model = "wflow",
    start_time = '1991-01-01T00:00:00Z',
    end_time = '1991-12-31T00:00:00Z',
    shape = None,
    forcing_info = dict(
        # Additional information about the external forcing data needed for the model configuration
        netcdfinput = "inmaps.nc",
        Precipitation = "/P",
        EvapoTranspiration = "/PET",
        Temperature = "/TEMP"
    )
)
print(forcing)

Forcing data for Wflow
----------------------
Directory: /home/peter/ewatercycle/ewatercycle/docs/examples/parameter-sets/wflow_rhine_sbm_nc
Start time: 1991-01-01T00:00:00Z
End time: 1991-12-31T00:00:00Z
Shapefile: None
Additional information for model config:
  - netcdfinput: inmaps.nc
  - Precipitation: /P
  - Temperature: /TEMP
  - EvapoTranspiration: /PET
  - Inflow: None


As you can see, the forcing consists of a generic part which is the same for all eWaterCycle models, and a model-specific part (`forcing_info`). If you're familiar with wflow, you might recognize that the model-specific settings map directly to wflow configuration settings. 

### Generating forcing data

In most cases, you will not have access to tailor-made forcing data, and manually pre-processing existing datasets can be quite a pain. eWaterCycle includes a forcing generator that can do all the required steps to go from the available datasets (ERA5, ERA-Interim, etc) to whatever format the models require. This is done through [ESMValTool recipes](https://docs.esmvaltool.org/en/latest/recipes/recipe_hydrology.html). For some models (e.g. lisflood) additional computations are done, as some steps require data and/or code that is not available to ESMValTool.

Apart from some standard parameters (start time, datasets, etc.), the forcing generator sometimes requires additional model-specific options. For our wflow example case, we need to pass the DEM file to the ESMValTool recipe as well. All model-specific options are listed in the [API documentation](https://ewatercycle.readthedocs.io/en/latest/apidocs/ewatercycle.forcing.html#ewatercycle.forcing.generate).

<div class="alert alert-info">
<p>ESMValTool configuration</p>
    
<p>As eWaterCycle relies on ESMValTool for processing forcing data, configuration for forcing is mostly defered to the esmvaltool configuration file. What ESMValTool configuration file to use can be specified in the <a href=https://ewatercycle.readthedocs.io/en/latest/system_setup.html#configure-esmvaltool>system setup</a>.</p>
</div>



In [3]:
forcing = ewatercycle.forcing.generate(
    target_model='wflow', 
    dataset='ERA5',
    start_time="1990-01-01T00:00:00Z",
    end_time="1990-01-31T00:00:00Z",
    shape='./examples/data/Rhine/Rhine.shp',
    model_specific_options = {
        'dem_file' : './examples/wflow_rhine_sbm_nc/staticmaps/wflow_dem.map',
    }
)
print(forcing)

Forcing data for Wflow
----------------------
Directory: /home/sarah/GitHub/ewatercycle/docs/examples/recipe_wflow_20210719_145723/work/wflow_daily/script
Start time: 1990-01-01T00:00:00Z
End time: 1990-01-31T00:00:00Z
Shapefile: /home/sarah/GitHub/ewatercycle/docs/examples/data/Rhine/Rhine.shp
Additional information for model config:
  - netcdfinput: wflow_ERA5_Rhine_1990_1990.nc
  - Precipitation: /pr
  - Temperature: /tas
  - EvapoTranspiration: /pet
  - Inflow: None


Generated forcing is automatically saved to the ESMValTool output directory. A `yaml` file is stored there as well, such that you can easily reload the forcing later without having to generate it again.

`ewatercycle_forcing.yaml`:

```yaml
!WflowForcing
start_time: '1990-01-01T00:00:00Z'
end_time: '1990-12-31T00:00:00Z'
shape:
netcdfinput: wflow_ERA5_Rhine_1990_1990.nc
Precipitation: /pr
EvapoTranspiration: /pet
Temperature: /tas
Inflow:
```

In [ ]:
reloaded_forcing = ewatercycle.forcing.load(
    directory='/home/sarah/GitHub/ewatercycle/docs/examples/recipe_wflow_20210719_145723/work/wflow_daily/script'
)

## Models

In [4]:
import ewatercycle.models

eWaterCycle currently integrates the following models:

* [wflow](https://ewatercycle.readthedocs.io/en/latest/examples/wflow.html)
* [pcrglobwb](https://ewatercycle.readthedocs.io/en/latest/examples/pcrglobwb.html)
* [marrmot M01](https://ewatercycle.readthedocs.io/en/latest/examples/marrmotm01.html)
* [marrmot M14](https://ewatercycle.readthedocs.io/en/latest/examples/marrmotm14.html)
* [lisflood](https://ewatercycle.readthedocs.io/en/latest/examples/lisflood.html)

and we're expecting to add more models soon. The process for adding new models is documented in [Adding models](https://ewatercycle.readthedocs.io/en/latest/adding_models.html)

### Model versions
To help with reproducibility the version of a model must always be specified when creating a model instance. The available versions can be seen like so:

In [3]:
import ewatercycle.models

ewatercycle.models.Wflow.available_versions

('2020.1.1',)

### Creating, setting up, and initializing a model instance

The way models are created, setup, and initialized matches [PyMT](https://pymt.readthedocs.io/en/latest/quickstart.html#run-a-model) as much as possible.
There are three steps:

- instantiate (create a python object that represents the model)
- setup (create a container with the right model, directories, and configuration files)
- initialize (start the model inside the container)

To a new user, these steps can be confusing as they seem to be related to "starting a model". However, you will see that there are some useful things that we can do in between each of these steps. As a side effect, splitting these steps also makes it easier to run a lot of models in parallel (e.g. for calibration). Experience tells us that you will quickly get used to it. 

When a model instance is created, we have to specify the version and pass in a suitable parameter set and forcing. 

In [ ]:
model_instance = ewatercycle.models.Wflow(version='2020.1.1', parameter_set=parameter_set, forcing=forcing)

In some specific cases the parameter set (e.g. for marrmot) or the forcing (e.g. when it is already included in the parameter set) is not needed.

Most models have a variety of parameters that can be set. An opiniated subset of these parameters is exposed through the eWaterCycle API. We focus on those settings that are relevant from a scientific point of view and prefer to hide technical settings. These parameters and their default values can be inspected as follows:

In [ ]:
model_instance.parameters

The start date and end date are automatically set based on the forcing data. 

Alternative values for each of these parameters can be passed on to the setup function:

In [ ]:
cfg_file, cfg_dir = model_instance.setup(
    end_time="1990-12-15T00:00:00Z"
)

The `setup` function does the following:

- Create a config directory which serves as the current working directory for the mode instance
- Creates a configuration file in this directory based on the settings
- Starts a container with the requested model version and access to the forcing and parameter sets.
- Input is mounted read-only, the working directory is mounted read-write (if a model cannot cope with inputs outside the working directory, the input will be copied).
- Setup will complain about incompatible model version, parameter_set, and forcing.

After `setup` but before `initialize` everything is good-to-go, but nothing has been done yet. This is
an opportunity to inspect the generated configuration file, and make any changes manually that could not be
done through the setup method. 

To modify the config file: print the path, open it in an editor, and save:

In [ ]:
print(cfg_file)

Once you're happy with the setup, it is time to initialize the model. You'll have to pass in the config file, even if you've not made any changes:

In [ ]:
model_instance.initialize(cfg_file)  # for some models, this step can take some time

### Running (and interacting with) a model

A model instance can be controlled by calling functions for running a single timestep (`update`), setting variables, and getting variables. Besides the rather lowlevel BMI functions like `get_value` and `set_value`, we also added convenience functions such as `get_value_as_xarray`, `get_value_at_coords`, `time_as_datetime`, and `time_as_isostr`. These make it even more pleasant to interact with the model.

For example, to run our model instance from start to finish, fetching the value of variable `discharge` at the location of a grdc station:

In [ ]:
grdc_latitude = ... # todo run observation cells below on system with GRDC data
grdc_longitude = ...

In [ ]:
output = []
while model_instance.time < model_instance.end_time:
    model_instance.update()
    
    discharge = model_instance.get_value_at_coords('RiverRunoff', lon=[grdc_longitude], lat=[grdc_latitudes])[0]
    output.append(discharge)
    
    # Here you could do whatever you like, e.g. update soil moisture values before doing the next timestep.
    
    print(model_instance.time_as_isostr, end="\r")  # "\r" clears the output before printing the next timestamp

We can also get the entire model field at a single time step. To simply plot it:

In [ ]:
model_instance.get_value_as_xarray('RiverRunoff').plot()

If you want to know which variables are available, you can use

In [ ]:
model_instance.output_var_names

## Observations

In [9]:
import ewatercycle.observation.grdc

This function automatically fetches the location of the GRDC data from the configuration file.
Start and end dates are fetched from the model instance

In [ ]:
grdc_station_id = 6335020

observations = ewatercycle.observation.grdc.get_grdc_data(
    station_id=grdc_station_id,
    start_time="1990-01-01T00:00:00Z", # model_instance.start_time_as_isostr
    end_time="1990-12-31T00:00:00Z"
)

observations.head()

In [ ]:
#Combine simulated and observated discharge into a single dataframe

import pandas
 
simulated_discharge_df = pandas.DataFrame(
    {'simulation': model_output}, index=timestamps
)
observations_df = observations.streamflow.to_dataframe().rename(
    columns={'streamflow': 'observation'}
)
discharge = simulated_discharge_df.join(observations_df)
discharge

#table with simulated and observed discharge

## Analysis
Once a model has run we can analyse the result. For this example we will assume a DataFrame was created with values over time for a certain location for which GRDC station data is available

In [ ]:
#Plot hydrograph

import ewatercycle.analysis

#todo: also add forcing to this hydrograph in some smart way
ewatercycle.analysis.hydrograph(
    discharge=discharge,
    reference='observation',
)

#nice hydrograph

In [ ]:
#finalize the model.
#as a side effect also destroys the container
model_instance.finalize()